In [4]:
# import libraries
import os
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

# file I/O
IN_CSV = "Final_Project.csv"
VIS_DIR = Path("visuals")
VIS_DIR.mkdir(exist_ok=True)

# helper to standardize text fields           
def to_title(s: str) -> str:
    # title case with common acronym fixes
    if s is None: return ""
    t = str(s).title()
    return (t.replace("Gp", "GP")
             .replace("Usa", "USA")
             .replace("Uk", "UK")
             .replace("Fia", "FIA"))

# helper to save visuals
def save_visual(fig, filename_no_ext: str):

    png_path = VIS_DIR / f"{filename_no_ext}.png"
    html_path = VIS_DIR / f"{filename_no_ext}.html"
    try:
        fig.write_image(str(png_path), scale=2, width=1000, height=650)
        print(f"Saved: {png_path}")
    except Exception as e:
        fig.write_html(str(html_path), include_plotlyjs="cdn")
        print(f"Saved (HTML fallback): {html_path}  | Reason: {e.__class__.__name__}")

def load_data(file):
    # load the csv file
    df = pd.read_csv(file)
    print("Successfully Loaded '%s' (Rows=%d, Cols=%d)" % (file, len(df), df.shape[1]))
    return df

def strip_norm(s):
    # normalize strings: strip + collapse spaces
    if pd.isna(s): return s
    return " ".join(str(s).strip().split())

def title_keep_acronyms(s):
    # title-case but preserve common acronyms
    if pd.isna(s): return s
    t = str(s).title()
    return (t.replace("Gp", "GP").replace("Usa", "USA").replace("Uk", "UK"))

def impute_by_season_median(df, cols, season_col="Season"):
    # impute numeric columns using per-season median then global median
    out = df.copy()
    for c in [x for x in cols if x in out.columns]:
        if season_col in out.columns:
            out[c] = out.groupby(season_col)[c].transform(lambda s: s.fillna(s.median()))
        out[c] = out[c].fillna(out[c].median())
    return out

def impute_categorical_mode(df, cols):
    # impute categoricals with mode
    out = df.copy()
    for c in [x for x in cols if x in out.columns]:
        m = out[c].dropna().mode()
        if not m.empty: out[c] = out[c].fillna(m.iloc[0])
    return out

def clean_data(df):
    # drop duplicates
    df = df.copy().drop_duplicates()

    # standardize key text fields if present
    for c in ["TrackType","StrategyType","OpeningCompound","LastCompound","RaceName","CircuitName","Driver","DriverCode"]:
        if c in df.columns:
            df[c] = df[c].map(strip_norm).map(title_keep_acronyms)

    # convert numeric where expected
    num_cols = [
        "RacePoints","FinishPosition","GridPosition","NumStops","FirstPitLap","SecondPitLap",
        "AvgStintLength","Stint1Length","Stint2Length","Stint3Length",
        "AirTemp","TrackTemp","HumidityPct","WindSpeedKMH",
        "AvgPitStopTime","StopRelativePitLossSeconds","DriverAvgRelativePitLossSeconds",
        "LapTimeVariation"
    ]
    for c in [x for x in num_cols if x in df.columns]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # plausibility filters
    if "FinishPosition" in df.columns:
        df.loc[(df["FinishPosition"] < 1) | (df["FinishPosition"] > 20), "FinishPosition"] = np.nan
    if "RacePoints" in df.columns:
        df.loc[(df["RacePoints"] < 0) | (df["RacePoints"] > 26), "RacePoints"] = np.nan
    if "FirstPitLap" in df.columns:
        df.loc[df["FirstPitLap"] < 1, "FirstPitLap"] = np.nan
    if "GridPosition" in df.columns:
        df.loc[df["GridPosition"] < 1, "GridPosition"] = np.nan
    for c in ["Stint1Length","Stint2Length"]:
        if c in df.columns:
            df = df[df[c] != 0]  # drop impossible zero-length stints
    # allow Stint3Length == 0

    # impute where helpful (season median / mode)
    df = impute_by_season_median(df, [c for c in num_cols if c in df.columns], season_col="Season" if "Season" in df.columns else None)
    df = impute_categorical_mode(df, [c for c in ["TrackType","StrategyType","OpeningCompound","LastCompound"] if c in df.columns])

    # strict dropna for analysis/visual consistency
    df = df.dropna(how="any").reset_index(drop=True)

    # cast clean ints
    for c in ["FinishPosition","GridPosition","NumStops","FirstPitLap","SecondPitLap","Stint1Length","Stint2Length","Stint3Length"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").round(0).astype(int)

    return df

# summary and descriptive statistics
def summarize_data(df):
    print("\nSummary Statistics - Numerical")
    num = df.select_dtypes(include=[np.number])
    if not num.empty:
        summary = num.describe().T[['mean','50%','std','min','max']].rename(columns={'50%':'median'}).round(3)
        display(summary)

    print("\nSummary Statistics - Categorical")
    cat = df.select_dtypes(include=['object'])
    if not cat.empty:
        display(df.describe(include='object'))

    if all(c in df.columns for c in ["TrackType","RacePoints"]):
        print("Average Race Points By Track Type")
        display(df.groupby("TrackType")["RacePoints"].mean().round(2).sort_values(ascending=False).to_frame("AvgRacePoints"))

    if all(c in df.columns for c in ["StrategyType","RacePoints"]):
        print("Average Race Points By Strategy Type")
        display(df.groupby("StrategyType")["RacePoints"].mean().round(2).sort_values(ascending=False).to_frame("AvgRacePoints"))

    if all(c in df.columns for c in ["OpeningCompound","RacePoints"]):
        print("Average Race Points By Opening Compound")
        display(df.groupby("OpeningCompound")["RacePoints"].mean().round(2).sort_values(ascending=False).to_frame("AvgRacePoints"))

# plot visuals
def plot_points_by_tracktype(df):
    if not all(c in df.columns for c in ["TrackType","RacePoints"]): return
    tmp = df.groupby("TrackType")["RacePoints"].mean().reset_index()
    fig = px.bar(tmp, x="TrackType", y="RacePoints",
                 title="Average Race Points By Track Type",
                 labels={"RacePoints":"Avg Points"})
    fig.update_xaxes(tickangle=30)
    fig.show()
    save_visual(fig, "points_by_track_type")

def plot_points_vs_grid(df):
    if not all(c in df.columns for c in ["GridPosition","RacePoints","TrackType"]): return
    fig = px.scatter(df, x="GridPosition", y="RacePoints", color="TrackType",
                     title="Race Points Vs Grid Position (By Track Type)",
                     labels={"GridPosition":"Grid Position (1=Front)", "RacePoints":"Race Points"},
                     hover_data=[c for c in ["StrategyType","OpeningCompound","LastCompound"] if c in df.columns])
    fig.update_traces(marker=dict(size=7, opacity=0.7))
    fig.show()
    save_visual(fig, "points_vs_grid")

def plot_strategy_mix_by_track(df):
    if not all(c in df.columns for c in ["TrackType","StrategyType"]): return
    mix = (df.groupby(["TrackType","StrategyType"]).size().reset_index(name="count"))
    fig = px.bar(mix, x="TrackType", y="count", color="StrategyType",
                 barmode="relative", title="Strategy Mix By Track Type (Count Share)")
    fig.update_xaxes(tickangle=30)
    fig.show()
    save_visual(fig, "strategy_mix_by_track")

def plot_pit_loss_distribution(df):
    if "StopRelativePitLossSeconds" not in df.columns: return
    fig = px.histogram(df, x="StopRelativePitLossSeconds", nbins=40,
                       title="Pit Stop Relative Loss (Seconds) — Distribution",
                       labels={"StopRelativePitLossSeconds":"Relative Pit Loss (s)"},
                       marginal="box")
    fig.show()
    save_visual(fig, "pit_loss_distribution")

def plot_numeric_correlation(df):
    cols = [c for c in [
        "RacePoints","FinishPosition","GridPosition","NumStops","FirstPitLap","SecondPitLap",
        "AvgStintLength","Stint1Length","Stint2Length","Stint3Length",
        "AirTemp","TrackTemp","HumidityPct","WindSpeedKMH",
        "AvgPitStopTime","StopRelativePitLossSeconds","DriverAvgRelativePitLossSeconds","LapTimeVariation"
    ] if c in df.columns]
    if len(cols) < 2: return
    corr = df[cols].corr().round(2)
    fig = go.Figure(data=go.Heatmap(
        z=corr.values, x=corr.columns, y=corr.columns,
        colorscale="RdBu", reversescale=True, zmid=0,
        text=corr.values, hoverinfo="text"))
    fig.update_layout(title="Correlation Matrix — Key Strategy Features", width=900, height=720)
    fig.show()
    save_visual(fig, "correlation_matrix")

def plot_stint_profiles(df):
    # distribution of stint lengths by track type and stint number
    have = [c for c in ["Stint1Length","Stint2Length","Stint3Length","TrackType"] if c in df.columns]
    if len(have) < 2: return
    cols = [c for c in ["Stint1Length","Stint2Length","Stint3Length"] if c in df.columns]
    tmp = df[["TrackType", *cols]].melt(id_vars="TrackType", var_name="Stint", value_name="Length").dropna(subset=["Length"])
    fig = px.box(tmp, x="TrackType", y="Length", color="Stint",
                 title="Stint Length Distribution By Track Type",
                 labels={"Length":"Laps"})
    fig.update_xaxes(tickangle=30)
    fig.show()
    save_visual(fig, "stint_length_by_track_type")

def plot_environment_context(df):
    # environment vs strategy context
    if all(c in df.columns for c in ["TrackTemp","StrategyType"]):
        tmp = df.groupby("StrategyType")["TrackTemp"].median().reset_index()
        fig = px.bar(tmp, x="StrategyType", y="TrackTemp",
                     title="Track Temperature By Strategy Type (Median)")
        fig.show()
        save_visual(fig, "track_temp_by_strategy")
    if all(c in df.columns for c in ["AirTemp","OpeningCompound"]):
        tmp = df.groupby("OpeningCompound")["AirTemp"].median().reset_index()
        fig = px.bar(tmp, x="OpeningCompound", y="AirTemp",
                     title="Air Temperature By Opening Compound (Median)")
        fig.show()
        save_visual(fig, "air_temp_by_opening_compound")

# run pipeline

# load
df = load_data(IN_CSV)

# clean + transform; drop rows with missing values
df_clean = clean_data(df)

# summaries
summarize_data(df_clean)

# visuals that support the business problem context
plot_points_by_tracktype(df_clean)
plot_points_vs_grid(df_clean)
plot_strategy_mix_by_track(df_clean)
plot_pit_loss_distribution(df_clean)
plot_numeric_correlation(df_clean)
plot_stint_profiles(df_clean)
plot_environment_context(df_clean)

print("\nVisuals saved to:", VIS_DIR.resolve())

Successfully Loaded 'Final_Project.csv' (Rows=1182, Cols=63)

Summary Statistics - Numerical


,mean,median,std,min,max
Season,2021.480,2022.000,1.866,2018.000,2024.000
Year,2021.480,2022.000,1.866,2018.000,2024.000
FinishPosition,5.585,5.000,3.561,1.000,18.000
RacePoints,9.832,10.000,6.915,0.000,26.000
IsPodium,0.316,0.000,0.465,0.000,1.000
IsPointsFinish,0.829,1.000,0.377,0.000,1.000
Stop,1.717,2.000,0.875,1.000,5.000
PitLap,27.380,26.000,14.474,1.000,63.000
Stint,2.235,2.000,1.171,1.000,7.000
StintLength,18.366,18.000,10.502,1.000,67.000



Summary Statistics - Categorical


,Date,RaceName,RaceKey,CircuitName,Location,Country,Driver,DriverCode,Status,TireCompound,OpeningCompound,LastCompound,StrategyType,Stint1Compound,Stint2Compound,Stint3Compound,TrackType
count,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147
unique,126,30,30,27,27,24,4,4,12,5,5,4,4,5,5,5,5
top,13/09/2020,Bahrain Grand Prix,Bahrain Grand Prix,Bahrain International Circuit,Sakhir,Italy,Charles Leclerc,Lec,Finished,Medium,Medium,Soft,2-Stop,Medium,Hard,Hard,High-Degradation
freq,46,96,96,108,108,122,546,546,976,385,485,427,479,485,499,349,475


Average Race Points By Track Type


...

Saved: visuals\pit_loss_distribution.png


Saved: visuals\correlation_matrix.png


Saved: visuals\stint_length_by_track_type.png


Saved: visuals\track_temp_by_strategy.png


Saved: visuals\air_temp_by_opening_compound.png

Visuals saved to: C:\Users\aksha\Documents\MGMT-3140\FINAL-PROJECT\visuals
